In [109]:
import time

from selenium import webdriver


def find_apartment_urls():
    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    # driver = webdriver.Chrome('/Users/moonpeter/Desktop/Selenium/chromedriver')
    driver.get('https://www.dabangapp.com/search#/map')

    driver.find_element_by_name('keyword').send_keys('성수동1가')
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath(
        '/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div[2]/ul[2]/li/label/span[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apt_list = []
    for i in range(6):
        apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
        for url in apart_urls:
            apt_list.append(url.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)

    # 성수 2동 시작.
    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')

    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))
    driver.close()
    return apt_list


def find_urls():
    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    # driver = webdriver.Chrome('/Users/moonpeter/Desktop/Selenium/chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://www.dabangapp.com/search#/map')
    driver.find_element_by_name('keyword').send_keys('성수동1가')
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()

    time.sleep(2)
    url_all_list = []
    for i in range(0, 3):
        time.sleep(2)
        seongsoo_1dong_detail_list = driver.find_elements_by_xpath(
            "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_1dong_detail_list:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))

    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()
    time.sleep(2)
    seongsoo_2dong_detail_list = driver.find_elements_by_xpath(
        "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul[2]/li/div/a")
    for data in seongsoo_2dong_detail_list:
        url_all_list.append(data.get_attribute('href'))
    for i in range(0, 2):
        time.sleep(2)
        seongsoo_2dong_detail_list = driver.find_elements_by_xpath(
            "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_2dong_detail_list[2:]:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))
    driver.close()
    return url_all_list

In [110]:
import os
import re
import time
import urllib

from django.core.files import File
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException

from config.settings import MEDIA_ROOT
from posts.crawling.find_urls import find_apartment_urls, find_urls


In [111]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
# driver = webdriver.Chrome('/Users/moonpeter/Desktop/Selenium/chromedriver')

# 다방 성수동 매물 url
url_all_list = find_apartment_urls()
url_all_list += find_urls()

54
98


In [112]:
url_all_list

['https://www.dabangapp.com/room/5e992d31d9d3a15c88491696',
 'https://www.dabangapp.com/room/5e9efb251e60da5787550276',
 'https://www.dabangapp.com/room/5e9fd1f1a639766eb587c9fc',
 'https://www.dabangapp.com/room/5e9e60010f85d2751c0ff977',
 'https://www.dabangapp.com/room/5e9efb20350ca26a1cb134db',
 'https://www.dabangapp.com/room/5e992d30f30bdb6390449f94',
 'https://www.dabangapp.com/room/5e9e81dad2c65a036b6485a2',
 'https://www.dabangapp.com/room/5e9e97194b4b4e3044719f0c',
 'https://www.dabangapp.com/room/5e990448a2d17027c7d5a1be',
 'https://www.dabangapp.com/room/5ea109a2bcb6b164ec1a2691',
 'https://www.dabangapp.com/room/5e9e967c3a7d963044febfe6',
 'https://www.dabangapp.com/room/5e991a6056d768629737a828',
 'https://www.dabangapp.com/room/5e9e7a1a3275bd3dedfbadbf',
 'https://www.dabangapp.com/room/5e7c13ee116bff1a5c1b0fe5',
 'https://www.dabangapp.com/room/5e7ad24a5809285b43dd6156',
 'https://www.dabangapp.com/room/5e8d636cdc19e6397ec46d1e',
 'https://www.dabangapp.com/room/5e9818a